## Demo notebook

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os,sys,inspect
sys.path.insert(0,"..")

In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]=""

In [ ]:
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import gdown

In [ ]:
def resize_img(path):
    c = Image.open(path)
    c = c.resize((320, 512), Image.BICUBIC).convert('RGB')
    c.save(path)

## Resize cloth image

In [ ]:
p = "../custom_images/cloth2.jpg"
resize_img(p)

## Resize person image

In [ ]:
p = "../custom_images/person.png"
resize_img(p)

## Get masks of clothing

In [ ]:
%cd U-2-Net
import u2net_load
import u2net_run
u2net = u2net_load.model(model_name = 'u2netp')
%cd ..

Put cloth images in cloth folder

In [ ]:
# obtain the mask of C 
u2net_run.infer(u2net, '../mpv3d_example_custom/cloth/', '../mpv3d_example_custom/cloth-mask/')

## Get 2D pose of person

In [ ]:
# !wget http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/body_25/pose_iter_584000.caffemodel

In [ ]:
import cv2
import numpy as np
import os
import json

class general_pose_model(object):
    def __init__(self, modelpath):
        # Specify the model to be used
        #   Body25: 25 points
        #   COCO:   18 points
        #   MPI:    15 points
        self.inWidth = 368
        self.inHeight = 368
        self.threshold = 0.05
        self.pose_net = self.general_body25_model(modelpath)
        
    def general_body25_model (self, modelpath):
        self.num_points = 25
        self.point_pairs = [[1, 0], [1, 2], [1, 5], 
                            [2, 3], [3, 4], [5, 6], 
                            [6, 7], [0, 15], [15, 17], 
                            [0, 16], [16, 18], [1, 8],
                            [8, 9], [9, 10], [10, 11], 
                            [11, 22], [22, 23], [11, 24],
                            [8, 12], [12, 13], [13, 14], 
                            [14, 19], [19, 20], [14, 21]]
        prototxt = os.path.join (
            modelpath, 
            "pose_deploy.prototxt")
        caffemodel = os.path.join (
            modelpath, 
            "pose_iter_584000.caffemodel")
        coco_model = cv2.dnn.readNetFromCaffe (prototxt, caffemodel)

        return coco_model

    def predict(self, imgfile):
        img_cv2 = cv2.imread(imgfile)
        img_height, img_width, _ = img_cv2.shape
        inpBlob = cv2.dnn.blobFromImage(img_cv2, 
                                        1.0 / 255, 
                                        (self.inWidth, self.inHeight),
                                        (0, 0, 0), 
                                        swapRB=False, 
                                        crop=False)
        self.pose_net.setInput(inpBlob)
        self.pose_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.pose_net.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL)

        output = self.pose_net.forward()

        H = output.shape[2]
        W = output.shape[3]
        
        points = []
        for idx in range(self.num_points):
            probMap = output[0, idx, :, :] # confidence map.

            # Find global maxima of the probMap.
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

            # Scale the point to fit on the original image
            x = (img_width * point[0]) / W
            y = (img_height * point[1]) / H

            if prob > self.threshold:
                points.append(x)
                points.append(y)
                points.append(prob)
            else:
                points.append(0)
                points.append(0)
                points.append(0)

        return points

def generate_pose_keypoints(img_file, pose_file):

    modelpath = 'pose'
    pose_model = general_pose_model(modelpath)

    res_points = pose_model.predict(img_file)
    
    pose_data = {"version": 1,
                 "people":  [
                                {"pose_keypoints_2d": res_points}
                            ]
                }

    pose_keypoints_path = pose_file

    json_object = json.dumps(pose_data, indent = 4) 
  
    # Writing to sample.json 
    with open(pose_keypoints_path, "w") as outfile: 
        outfile.write(json_object) 
    print('File saved at {}'.format(pose_keypoints_path))

In [ ]:
# https://titanwolf.org/Network/Articles/Article?AID=d4b5fc84-c88e-4159-9a72-a63486d93ce2

In [ ]:
generate_pose_keypoints("/home/hz/m3d-vton/mpv3d_example_custom/image/ZX121DA0E-Q11@6=person_whole_front.png", "ZX121DA0E-Q11@6=person_whole_front_keypints.json")